In [1]:
import os
import time
import tensorflow as tf
from tensorflow.keras import models, layers, Model, callbacks
from tensorflow.keras.metrics import AUC, Precision, Recall
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from process_data import process_data
from nn_gridsearch import logdir

In [2]:
data = process_data()

In [3]:
for k, v in data.items():
    print(k)
    print(type(v))

x_train
<class 'numpy.ndarray'>
x_train_processed
<class 'numpy.ndarray'>
x_test
<class 'numpy.ndarray'>
x_test_processed
<class 'numpy.ndarray'>
y_train
<class 'numpy.ndarray'>
y_test
<class 'numpy.ndarray'>
pipeline
<class 'sklearn.pipeline.Pipeline'>


In [4]:
x_train_processed = data['x_train_processed']
y_train = data['y_train']

In [5]:
x_train_processed.shape

(1047, 11)

> note: old model constrution, using Sequential API

In [6]:
#model = models.Sequential()
#model.add(layers.Dense(32, activation='relu', input_shape=(x_train_processed.shape[1],)))
#model.add(layers.Dense(32, activation='relu'))
#model.add(layers.Dense(1, activation='sigmoid'))


> changed to functional API --> more flexibility, for future expansion

In [7]:
input_ = layers.Input(shape=x_train_processed.shape[1:])
hidden_0 = layers.Dense(80, activation='relu')(input_)
hidden_1 = layers.Dense(80, activation='relu')(hidden_0)
hidden_2 = layers.Dense(80, activation='relu')(hidden_1)
hidden_3 = layers.Dense(80, activation='relu')(hidden_2)
hidden_4 = layers.Dense(80, activation='relu')(hidden_3)
hidden_5 = layers.Dense(80, activation='relu')(hidden_4)
output = layers.Dense(1, activation='sigmoid')(hidden_5)
model_custom = Model(inputs=[input_], outputs=[output])

In [8]:
#model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model_custom.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), AUC()])

In [9]:
model_custom.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 11)]              0         
_________________________________________________________________
dense (Dense)                (None, 80)                960       
_________________________________________________________________
dense_1 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_2 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_3 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_4 (Dense)              (None, 80)                6480      
_________________________________________________________________
dense_5 (Dense)              (None, 80)                6480  

In [10]:
history = model_custom.fit(
    data['x_train_processed'], data['y_train'], validation_split=.2, epochs=300,
    callbacks=[
        EarlyStopping(patience=10, monitor='val_loss', mode='min'),
        TensorBoard(logdir('custom_model'))])

Epoch 1/300
27/27 [==============================] - 2s 42ms/step - loss: 0.5990 - accuracy: 0.6595 - precision: 0.5753 - recall: 0.1419 - auc: 0.7003 - val_loss: 0.6181 - val_accuracy: 0.7190 - val_precision: 0.6753 - val_recall: 0.6047 - val_auc: 0.7659
Epoch 2/300
27/27 [==============================] - 1s 20ms/step - loss: 0.4746 - accuracy: 0.8017 - precision: 0.7241 - recall: 0.7095 - auc: 0.8275 - val_loss: 0.5483 - val_accuracy: 0.7429 - val_precision: 0.7162 - val_recall: 0.6163 - val_auc: 0.7987
Epoch 3/300
27/27 [==============================] - 1s 21ms/step - loss: 0.4286 - accuracy: 0.8136 - precision: 0.7917 - recall: 0.6419 - auc: 0.8568 - val_loss: 0.4865 - val_accuracy: 0.7429 - val_precision: 0.7000 - val_recall: 0.6512 - val_auc: 0.8161
Epoch 4/300
27/27 [==============================] - 1s 21ms/step - loss: 0.4189 - accuracy: 0.8256 - precision: 0.8261 - recall: 0.6419 - auc: 0.8601 - val_loss: 0.4951 - val_accuracy: 0.7333 - val_precision: 0.6786 - val_recall: 0

In [11]:
%load_ext tensorboard
%tensorboard --logdir ./custom_logs

In [12]:
model_gridsearch = models.load_model('./models/titanic_gridsearch_best.h5')

In [13]:
model_gridsearch.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden_layer_0_relu_alpha_0. (None, 31)                372       
_________________________________________________________________
batch_normalization_8 (Batch (None, 31)                124       
_________________________________________________________________
activation_8 (Activation)    (None, 31)                0         
_________________________________________________________________
dropout_0_30 (Dropout)       (None, 31)                0         
_________________________________________________________________
hidden_layer_1_relu_alpha_0. (None, 31)                992       
_________________________________________________________________
batch_normalization_9 (Batch (None, 31)                124       
_________________________________________________________________
activation_9 (Activation)    (None, 31)               

In [14]:
model_custom.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 0s 17ms/step - loss: 0.4881 - accuracy: 0.7748 - precision: 0.8242 - recall: 0.6356 - auc: 0.8577


[0.4881334900856018,
 0.7748091816902161,
 0.8241758346557617,
 0.6355932354927063,
 0.8576683402061462]

In [15]:
model_gridsearch.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 1s 4ms/step - loss: 0.5056 - accuracy: 0.7710 - precision_1: 0.8816 - recall_1: 0.5678 - auc_1: 0.8524


[0.5055856704711914,
 0.7709923386573792,
 0.8815789222717285,
 0.5677965879440308,
 0.8524011373519897]

In [16]:
model_synthetic = models.load_model('./models/titanic_gridsearch_synthetic_2021_10_15_10_44.h5')
model_synthetic.evaluate(data['x_test_processed'], data['y_test'])

9/9 [==============================] - 2s 9ms/step - loss: 0.5042 - accuracy: 0.7634 - precision_1: 0.8684 - recall_1: 0.5593 - auc_1: 0.8453


[0.5042383670806885,
 0.7633587718009949,
 0.8684210777282715,
 0.5593220591545105,
 0.845280110836029]

> todo next - function to build model, sklearn wrapper from keras, pipelines, random grid search for model!